# 说明
这是一篇纯粹的例子。

作用是爬取百度贴吧的某个帖子的图片:
<img src='./image/tieba.png' />

In [1]:
from bs4 import BeautifulSoup
import requests
import contextlib
import sys
import os
import traceback

In [2]:
headers = {'Connection': "keep-alive",
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'zh-CN,zh;q=0.9'}

In [3]:
def Schedule(a, b, c):
    per = 100.0 * a * b / c
    if per > 100:
        per = 100
    print("  " + "%.2f%% 已经下载的大小:%ld 文件大小:%ld" % (per, a * b, c) + '\r')

In [4]:
def urlretrieve(url, filename=None, reporthook=None, params=None):
    print('download file: {0}'.format(url))
    '''传入ID改变url，利用closing跟iter_content下载图片'''
    with contextlib.closing(requests.get(url, stream=True, headers=headers, params=params)) as fp:  # 打开网页
        header = fp.headers
        with open(filename, 'wb+') as tfp:  # w是覆盖原文件，a是追加写入 #打开文件
            bs = 1024
            size = -1
            blocknum = 0
            if "content-length" in header:
                size = int(header["Content-Length"])  # 文件的总大小理论值
            if reporthook and size > 0:
                reporthook(blocknum, bs, size)  # 写入前运行一次回调函数

            for chunk in fp.iter_content(chunk_size=1024):
                if chunk:
                    tfp.write(chunk)  # 写入
                    tfp.flush()
                    blocknum += 1
                    if reporthook and size > 0:
                        reporthook(blocknum, bs, size)  # 每写入一次就运行一次回调函数

In [5]:
def crawl(url):
    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        imglist = soup.find_all('img', class_='BDE_Image')
        folder = './crawlfile/tieba'
        for index, imgtag in enumerate(imglist):
            print('download {0}/{1} files'.format(index+1, len(imglist)))
            imgurl = str(imgtag['src'])
            if imgurl.endswith('.jpg'):
                urlretrieve(url=imgurl,
                            filename=os.path.join(folder,
                                                  '{0}.jpg'.format(index+1)), reporthook=Schedule)


        print(r.encoding)
    except Exception as e:
        print(e)
        traceback.print_exc()

In [7]:
folder = './crawlfile/tieba'
if not os.path.exists(folder):
    os.makedirs(folder)
crawl('https://tieba.baidu.com/p/7297283378')

download 1/21 files
download file: http://tiebapic.baidu.com/forum/w%3D580/sign=7cd826f789510fb37819779fe932c893/aa13632762d0f7030b9afd921ffa513d2697c550.jpg
  0.00% 已经下载的大小:0 文件大小:130927
  0.78% 已经下载的大小:1024 文件大小:130927
  1.56% 已经下载的大小:2048 文件大小:130927
  2.35% 已经下载的大小:3072 文件大小:130927
  3.13% 已经下载的大小:4096 文件大小:130927
  3.91% 已经下载的大小:5120 文件大小:130927
  4.69% 已经下载的大小:6144 文件大小:130927
  5.47% 已经下载的大小:7168 文件大小:130927
  6.26% 已经下载的大小:8192 文件大小:130927
  7.04% 已经下载的大小:9216 文件大小:130927
  7.82% 已经下载的大小:10240 文件大小:130927
  8.60% 已经下载的大小:11264 文件大小:130927
  9.39% 已经下载的大小:12288 文件大小:130927
  10.17% 已经下载的大小:13312 文件大小:130927
  10.95% 已经下载的大小:14336 文件大小:130927
  11.73% 已经下载的大小:15360 文件大小:130927
  12.51% 已经下载的大小:16384 文件大小:130927
  13.30% 已经下载的大小:17408 文件大小:130927
  14.08% 已经下载的大小:18432 文件大小:130927
  14.86% 已经下载的大小:19456 文件大小:130927
  15.64% 已经下载的大小:20480 文件大小:130927
  16.42% 已经下载的大小:21504 文件大小:130927
  17.21% 已经下载的大小:22528 文件大小:130927
  17.99% 已经下载的大小:23552 文件大小:130927
  18.77% 已经下载的大小:24576 文件大小:

  0.00% 已经下载的大小:0 文件大小:83780
  1.22% 已经下载的大小:1024 文件大小:83780
  2.44% 已经下载的大小:2048 文件大小:83780
  3.67% 已经下载的大小:3072 文件大小:83780
  4.89% 已经下载的大小:4096 文件大小:83780
  6.11% 已经下载的大小:5120 文件大小:83780
  7.33% 已经下载的大小:6144 文件大小:83780
  8.56% 已经下载的大小:7168 文件大小:83780
  9.78% 已经下载的大小:8192 文件大小:83780
  11.00% 已经下载的大小:9216 文件大小:83780
  12.22% 已经下载的大小:10240 文件大小:83780
  13.44% 已经下载的大小:11264 文件大小:83780
  14.67% 已经下载的大小:12288 文件大小:83780
  15.89% 已经下载的大小:13312 文件大小:83780
  17.11% 已经下载的大小:14336 文件大小:83780
  18.33% 已经下载的大小:15360 文件大小:83780
  19.56% 已经下载的大小:16384 文件大小:83780
  20.78% 已经下载的大小:17408 文件大小:83780
  22.00% 已经下载的大小:18432 文件大小:83780
  23.22% 已经下载的大小:19456 文件大小:83780
  24.44% 已经下载的大小:20480 文件大小:83780
  25.67% 已经下载的大小:21504 文件大小:83780
  26.89% 已经下载的大小:22528 文件大小:83780
  28.11% 已经下载的大小:23552 文件大小:83780
  29.33% 已经下载的大小:24576 文件大小:83780
  30.56% 已经下载的大小:25600 文件大小:83780
  31.78% 已经下载的大小:26624 文件大小:83780
  33.00% 已经下载的大小:27648 文件大小:83780
  34.22% 已经下载的大小:28672 文件大小:83780
  35.45% 已经下载的大小:29696 文件大小:83780
  

  0.00% 已经下载的大小:0 文件大小:26566
  3.85% 已经下载的大小:1024 文件大小:26566
  7.71% 已经下载的大小:2048 文件大小:26566
  11.56% 已经下载的大小:3072 文件大小:26566
  15.42% 已经下载的大小:4096 文件大小:26566
  19.27% 已经下载的大小:5120 文件大小:26566
  23.13% 已经下载的大小:6144 文件大小:26566
  26.98% 已经下载的大小:7168 文件大小:26566
  30.84% 已经下载的大小:8192 文件大小:26566
  34.69% 已经下载的大小:9216 文件大小:26566
  38.55% 已经下载的大小:10240 文件大小:26566
  42.40% 已经下载的大小:11264 文件大小:26566
  46.25% 已经下载的大小:12288 文件大小:26566
  50.11% 已经下载的大小:13312 文件大小:26566
  53.96% 已经下载的大小:14336 文件大小:26566
  57.82% 已经下载的大小:15360 文件大小:26566
  61.67% 已经下载的大小:16384 文件大小:26566
  65.53% 已经下载的大小:17408 文件大小:26566
  69.38% 已经下载的大小:18432 文件大小:26566
  73.24% 已经下载的大小:19456 文件大小:26566
  77.09% 已经下载的大小:20480 文件大小:26566
  80.95% 已经下载的大小:21504 文件大小:26566
  84.80% 已经下载的大小:22528 文件大小:26566
  88.65% 已经下载的大小:23552 文件大小:26566
  92.51% 已经下载的大小:24576 文件大小:26566
  96.36% 已经下载的大小:25600 文件大小:26566
  100.00% 已经下载的大小:26624 文件大小:26566
download 9/21 files
download file: http://tiebapic.baidu.com/forum/w%3D580/sign=a1b1bf419094a4c20

  0.00% 已经下载的大小:0 文件大小:67435
  1.52% 已经下载的大小:1024 文件大小:67435
  3.04% 已经下载的大小:2048 文件大小:67435
  4.56% 已经下载的大小:3072 文件大小:67435
  6.07% 已经下载的大小:4096 文件大小:67435
  7.59% 已经下载的大小:5120 文件大小:67435
  9.11% 已经下载的大小:6144 文件大小:67435
  10.63% 已经下载的大小:7168 文件大小:67435
  12.15% 已经下载的大小:8192 文件大小:67435
  13.67% 已经下载的大小:9216 文件大小:67435
  15.18% 已经下载的大小:10240 文件大小:67435
  16.70% 已经下载的大小:11264 文件大小:67435
  18.22% 已经下载的大小:12288 文件大小:67435
  19.74% 已经下载的大小:13312 文件大小:67435
  21.26% 已经下载的大小:14336 文件大小:67435
  22.78% 已经下载的大小:15360 文件大小:67435
  24.30% 已经下载的大小:16384 文件大小:67435
  25.81% 已经下载的大小:17408 文件大小:67435
  27.33% 已经下载的大小:18432 文件大小:67435
  28.85% 已经下载的大小:19456 文件大小:67435
  30.37% 已经下载的大小:20480 文件大小:67435
  31.89% 已经下载的大小:21504 文件大小:67435
  33.41% 已经下载的大小:22528 文件大小:67435
  34.93% 已经下载的大小:23552 文件大小:67435
  36.44% 已经下载的大小:24576 文件大小:67435
  37.96% 已经下载的大小:25600 文件大小:67435
  39.48% 已经下载的大小:26624 文件大小:67435
  41.00% 已经下载的大小:27648 文件大小:67435
  42.52% 已经下载的大小:28672 文件大小:67435
  44.04% 已经下载的大小:29696 文件大小:67435


  0.00% 已经下载的大小:0 文件大小:588583
  0.17% 已经下载的大小:1024 文件大小:588583
  0.35% 已经下载的大小:2048 文件大小:588583
  0.52% 已经下载的大小:3072 文件大小:588583
  0.70% 已经下载的大小:4096 文件大小:588583
  0.87% 已经下载的大小:5120 文件大小:588583
  1.04% 已经下载的大小:6144 文件大小:588583
  1.22% 已经下载的大小:7168 文件大小:588583
  1.39% 已经下载的大小:8192 文件大小:588583
  1.57% 已经下载的大小:9216 文件大小:588583
  1.74% 已经下载的大小:10240 文件大小:588583
  1.91% 已经下载的大小:11264 文件大小:588583
  2.09% 已经下载的大小:12288 文件大小:588583
  2.26% 已经下载的大小:13312 文件大小:588583
  2.44% 已经下载的大小:14336 文件大小:588583
  2.61% 已经下载的大小:15360 文件大小:588583
  2.78% 已经下载的大小:16384 文件大小:588583
  2.96% 已经下载的大小:17408 文件大小:588583
  3.13% 已经下载的大小:18432 文件大小:588583
  3.31% 已经下载的大小:19456 文件大小:588583
  3.48% 已经下载的大小:20480 文件大小:588583
  3.65% 已经下载的大小:21504 文件大小:588583
  3.83% 已经下载的大小:22528 文件大小:588583
  4.00% 已经下载的大小:23552 文件大小:588583
  4.18% 已经下载的大小:24576 文件大小:588583
  4.35% 已经下载的大小:25600 文件大小:588583
  4.52% 已经下载的大小:26624 文件大小:588583
  4.70% 已经下载的大小:27648 文件大小:588583
  4.87% 已经下载的大小:28672 文件大小:588583
  5.05% 已经下载的大小:29696 文件大小:

  0.00% 已经下载的大小:0 文件大小:51266
  2.00% 已经下载的大小:1024 文件大小:51266
  3.99% 已经下载的大小:2048 文件大小:51266
  5.99% 已经下载的大小:3072 文件大小:51266
  7.99% 已经下载的大小:4096 文件大小:51266
  9.99% 已经下载的大小:5120 文件大小:51266
  11.98% 已经下载的大小:6144 文件大小:51266
  13.98% 已经下载的大小:7168 文件大小:51266
  15.98% 已经下载的大小:8192 文件大小:51266
  17.98% 已经下载的大小:9216 文件大小:51266
  19.97% 已经下载的大小:10240 文件大小:51266
  21.97% 已经下载的大小:11264 文件大小:51266
  23.97% 已经下载的大小:12288 文件大小:51266
  25.97% 已经下载的大小:13312 文件大小:51266
  27.96% 已经下载的大小:14336 文件大小:51266
  29.96% 已经下载的大小:15360 文件大小:51266
  31.96% 已经下载的大小:16384 文件大小:51266
  33.96% 已经下载的大小:17408 文件大小:51266
  35.95% 已经下载的大小:18432 文件大小:51266
  37.95% 已经下载的大小:19456 文件大小:51266
  39.95% 已经下载的大小:20480 文件大小:51266
  41.95% 已经下载的大小:21504 文件大小:51266
  43.94% 已经下载的大小:22528 文件大小:51266
  45.94% 已经下载的大小:23552 文件大小:51266
  47.94% 已经下载的大小:24576 文件大小:51266
  49.94% 已经下载的大小:25600 文件大小:51266
  51.93% 已经下载的大小:26624 文件大小:51266
  53.93% 已经下载的大小:27648 文件大小:51266
  55.93% 已经下载的大小:28672 文件大小:51266
  57.93% 已经下载的大小:29696 文件大小:51266